<a href="https://colab.research.google.com/github/priyanka36/Inferencing-on-Edge-Devices/blob/main/PostTrainingQuantization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!nvidia-smi


Thu Jul  1 08:55:16 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P8    12W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [12]:
from __future__ import absolute_import,division,print_function,unicode_literals

try:
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
import os
import tensorflow_datasets as tfds

%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [10]:
!ls -alrt '/content/drive/My Drive/models/'

total 19287
-rw------- 1 root root 19749728 Jul  1 07:50 final_fmnist_model.hdf5


In [15]:
model = tf.keras.models.load_model('/content/drive/My Drive/models/final_fmnist_model.hdf5')
print(model.summary())

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 28, 28, 64)        320       
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 14, 14, 64)        0         
_________________________________________________________________
dropout_16 (Dropout)         (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 14, 14, 128)       32896     
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 7, 7, 128)         0         
_________________________________________________________________
dropout_17 (Dropout)         (None, 7, 7, 128)         0         
_________________________________________________________________
flatten_8 (Flatten)          (None, 6272)             

In [6]:
pwd

'/content'

In [16]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmp58bkqu95/assets


In [21]:
model_size = len(tflite_model)/1024
print("model size = %dKBs "% model_size)

model size = 6416KBs 


In [23]:
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_LATENCY]
tflite_quantized_model_1 = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpnagts48o/assets


INFO:tensorflow:Assets written to: /tmp/tmpnagts48o/assets


In [27]:
(x_train,y_train),(x_test,y_test) = tf.keras.datasets.fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [28]:
x_test.shape

(10000, 28, 28)

In [32]:


# converter.target_spec.supported_ops=[tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# converter.inference_input_type = tf.uint8
# converter.inference_output_type = tf.uint8
# #For int 8 we need representative dataset
# tflite_quantized_model_2 = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmp9xdua2ds/assets


INFO:tensorflow:Assets written to: /tmp/tmp9xdua2ds/assets


ValueError: ignored

In [33]:
x_test.shape

(10000, 28, 28)

In [36]:
import numpy as np
x_test = np.expand_dims(x_test,-1)



In [37]:
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()
input_tensor_index = interpreter.get_input_details()[0]["index"]
output = interpreter.tensor(interpreter.get_output_details()[0]["index"])

In [38]:
interpreter.get_tensor_details()

[{'dtype': numpy.float32,
  'index': 0,
  'name': 'conv2d_16_input',
  'quantization': (0.0, 0),
  'quantization_parameters': {'quantized_dimension': 0,
   'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32)},
  'shape': array([ 1, 28, 28,  1], dtype=int32),
  'shape_signature': array([-1, 28, 28,  1], dtype=int32),
  'sparsity_parameters': {}},
 {'dtype': numpy.float32,
  'index': 1,
  'name': 'sequential_8/conv2d_17/BiasAdd;sequential_8/conv2d_17/Conv2D;sequential_8/conv2d_17/BiasAdd/ReadVariableOp/resource',
  'quantization': (0.0, 0),
  'quantization_parameters': {'quantized_dimension': 0,
   'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32)},
  'shape': array([128], dtype=int32),
  'shape_signature': array([128], dtype=int32),
  'sparsity_parameters': {}},
 {'dtype': numpy.float32,
  'index': 2,
  'name': 'sequential_8/conv2d_16/BiasAdd;sequential_8/conv2d_16/Conv2D;sequential_8/conv2d_16/BiasAdd/ReadVariableOp/resource',
  'quan

In [39]:
all_layers = interpreter.get_tensor_details()
for layer in all_layers:
  print(interpreter.get_tensor(layer['index']))

[[[[0.00000000e+00]
   [0.00000000e+00]
   [1.14794370e-41]
   [0.00000000e+00]
   [2.98286150e+13]
   [3.06940416e-41]
   [1.37607509e-42]
   [0.00000000e+00]
   [6.27641582e-42]
   [0.00000000e+00]
   [1.68242618e-37]
   [1.74537383e-33]
   [           nan]
   [           nan]
   [1.45446220e-30]
   [1.47341928e-32]
   [           nan]
   [1.44181715e-38]
   [1.43241030e-33]
   [           nan]
   [4.34544809e-31]
   [           nan]
   [           nan]
   [2.93498641e-35]
   [           nan]
   [           nan]
   [4.13259593e-39]
   [2.54282291e-32]]

  [[3.01220399e-38]
   [           nan]
   [           nan]
   [1.26340993e-31]
   [           nan]
   [           nan]
   [5.96576022e-30]
   [1.21342219e-32]
   [7.12877019e-38]
   [5.67140477e-30]
   [           nan]
   [6.98353272e-29]
   [6.34832589e-31]
   [           nan]
   [           nan]
   [1.26389141e-34]
   [           nan]
   [           nan]
   [           nan]
   [1.41555836e-32]
   [           nan]
   [1.78205884e-32

In [ ]:
prediction_output=[]

for test_image in x_test:

  test_image = np.expand_dims(test_image,axis=0).astype(np.float)
  interpreter.set_tensor(input_tensor_index,test_image)
  interpreter.invoke()
  out = np.argmax(output()[0])
  prediction_output.append(out)
  

In [ ]:
accurate_count = 0
for index in range(len(prediction_output)):
  if prediction_output[index] == y_test[index]
  accurate_count +=1
accuracy = accurate_count*1.0/len(prediction_output)

In [ ]:
index = 0
for test_image in x_test[:10]:
  img = np.expand_dims(test_image,axis=0)
  